## Popularity-based model
Мы будем работать с набором данных Movie Lens. Он содержит идентификаторы для каждого фильма и пользователя, который его смотрел, а также оценку, которую пользователь поставил фильму. В датасете представлено 25 000 095 оценок фильмов от 162 541 пользователя со шкалой оценок от 0.5 до 5.0.

Из этого набора нам понадобится два файла:

* данные о фильмах (movies);
* данные о выставленных оценках (ratings).
Объединим их:

In [39]:
import pandas as pd
import numpy as np

In [2]:
ratings = pd.read_csv('data/dst-3.0_mathml_14_5_rating.csv')
movies = pd.read_csv('data/movie.zip')
df=pd.merge(ratings, movies, how='left', on='movieId')
df.head()


,userId,movieId,rating,timestamp,title,genres
0,1,2,3.5,2005-04-02 23:53:47,Jumanji (1995),Adventure|Children|Fantasy
1,1,29,3.5,2005-04-02 23:31:16,"City of Lost Children, The (Cité des enfants p...",Adventure|Drama|Fantasy|Mystery|Sci-Fi
2,1,32,3.5,2005-04-02 23:33:39,Twelve Monkeys (a.k.a. 12 Monkeys) (1995),Mystery|Sci-Fi|Thriller
3,1,47,3.5,2005-04-02 23:32:07,Seven (a.k.a. Se7en) (1995),Mystery|Thriller
4,1,50,3.5,2005-04-02 23:29:40,"Usual Suspects, The (1995)",Crime|Mystery|Thriller


* userId — id пользователя;
* movieId — id фильма;
* rating — выставленный пользователем рейтинг для фильма;
* timestamp — время выставления рейтинга;
* title — название фильма;
* genres — жанры, к которым относится фильм.

In [3]:
df['title'].value_counts().head(5)

Pulp Fiction (1994)                 67310
Forrest Gump (1994)                 66172
Shawshank Redemption, The (1994)    63366
Silence of the Lambs, The (1991)    63299
Jurassic Park (1993)                59715
Name: title, dtype: int64

Отлично, мы нашли самые востребованные фильмы. Однако если фильм посмотрело много людей, это ещё не значит, что он им понравился. Чтобы понять, как зритель на самом деле относится к фильму, нужны более чёткие данные. К счастью, в наборе данных Movie Lens есть оценки каждого из зрителей.

* Найдите средний рейтинг для каждого из фильмов.
* Найдите фильмы с наивысшим средним рейтингом.
* Введите в качестве ответа фильм, занимающий последнее место среди фильмов с наивысшим рейтингом, если предварительно отсортировать их по алфавитному порядку.


In [9]:
mean_rating = df.groupby(by='title', as_index=False)['rating'].mean().sort_values(by=['rating', 'title'], ascending=[False, True])
mean_rating[mean_rating['rating'] == 5].tail()

,title,rating
25607,Welcome to Australia (1999),5.0
25751,When I Walk (2013),5.0
25918,Who Killed Vincent Chin? (1987),5.0
26430,Year Zero: The Silent Death of Cambodia (1979),5.0
26482,Yonkers Joe (2008),5.0


В двух предыдущих заданиях вы использовали два самых простых метода для создания неперсонализированных рекомендаций. Однако у них обоих есть свои недостатки: поиск наиболее часто просматриваемых фильмов не учитывает того, насколько фильм нравится аудитории, а поиск среднего рейтинга может вывести в рекомендуемые фильмы малоизвестные специфические картины с одной-двумя оценками.

Чтобы решить эти проблемы, объединим два подхода и будем искать средний рейтинг только для фильмов, которые были оценены более 50 раз.

In [15]:
rating_count_title = df.groupby(
    by='title', 
    as_index=False)[[
    'rating', 
    'movieId'
]].agg({
    'rating': 'mean', 
    'movieId': 'count'
}).rename(columns={
    'rating': 'mean_rating',
    'movieId': 'count'
})
rating_count_title = rating_count_title[rating_count_title['count'] > 50]
rating_count_title.shape[0]

10472

Постройте простейшую рекомендацию: возьмите фильмы, которые смотрели более 50 раз, и найдите среди них фильм с наивысшей средней оценкой.

In [17]:
top1_film = rating_count_title.sort_values(by='mean_rating', ascending=False).head(1)
top1_film

,title,mean_rating,count
20779,"Shawshank Redemption, The (1994)",4.44699,63366


***
В датасете содержится около 73 тысяч записей о взаимодействии пользователей с более чем тремя тысячами публичных статей, размещённых на платформе.

Информация в наборе данных:
* Оригинальный URL, название и текст статьи.
* Контекст посещений пользователей, например дата/время, клиент (мобильное приложение/браузер) и геолокация.
* Различные типы взаимодействия, что позволяет сделать вывод об уровне заинтересованности пользователя в статьях, например комментарии → лайки → просмотры.

Данные включают в себя два файла:

* `shared_articles.csv`;
* `users_interactions.csv`.

Начнём работать с файлом `shared_articles.csv`. Он содержит информацию о статьях, опубликованных на платформе *DeskDrop*.

Для каждой статьи есть:
* дата публикации (временная метка),
* исходный URL-адрес,
* заголовок,
* содержание в виде обычного текста,
* язык статьи (португальский — pt или английский — en),
* информация о пользователе, который поделился статьёй (автор).

Для временной метки существует два возможных типа событий:
* *CONTENT SHARED* — статья была опубликована на платформе и доступна для пользователей;
* *CONTENT REMOVED* — статья была удалена с платформы и недоступна для дальнейших рекомендаций.

Для простоты мы рассматриваем здесь только тип события *CONTENT SHARED*.

In [20]:
articles_data = pd.read_csv('data/shared_articles.csv')
articles_data.head(1)

,timestamp,eventType,contentId,authorPersonId,authorSessionId,authorUserAgent,authorRegion,authorCountry,contentType,url,title,text,lang
0,1459192779,CONTENT REMOVED,-6451309518266745024,4340306774493623681,8940341205206233829,NaN,NaN,NaN,HTML,http://www.nytimes.com/2016/03/28/business/dea...,"Ethereum, a Virtual Currency, Enables Transact...",All of this work is still very early. The firs...,en


Отфильтруйте данные так, чтобы остались только объекты с типом события *CONTENT SHARED*. Сколько таких объектов в получившейся таблице?

In [21]:
articles_data = articles_data[articles_data['eventType'] == 'CONTENT SHARED']
articles_data.shape[0]

3047

Теперь откроем второй файл — `users_interactions.csv`.

Предварительно преобразуем столбцы `personId`, `contentId` в таблицах к строкам. 

In [22]:
interactions_data = pd.read_csv('data/users_interactions.csv')
interactions_data.personId = interactions_data.personId.astype(str)
interactions_data.contentId = interactions_data.contentId.astype(str)
articles_data.contentId = articles_data.contentId.astype(str)

В колонке `eventType` описаны действия, которые могли совершать пользователи при взаимодействии со статьёй:
* VIEW — просмотр,
* LIKE — лайк,
* COMMENT CREATED — комментарий,
* FOLLOW — подписка,
* BOOKMARK — добавление в закладки.

В первую очередь нам необходимо понять, как определить, что какая-то статья популярнее других. Если бы из возможных реакций у нас были только лайки или только просмотры, то статьи было бы легко ранжировать в соответствии с этими значениями. Однако у нас есть информация о различных действиях пользователя, и на её основе мы должны создать некий универсальный индекс популярности. Составим его из реакций пользователей, придав им разные веса:

In [23]:
event_type = {
   'VIEW': 1.0,
   'LIKE': 2.0, 
   'BOOKMARK': 2.5, 
   'FOLLOW': 3.0,
   'COMMENT CREATED': 4.0,  
}

Веса здесь подобраны исходя из важности каждого действия: оставить комментарий — значит, показать наибольшую вовлечённость, а обычный просмотр, напротив, демонстрирует наименьшую вовлечённость.

Создайте признак, который будет отражать числовой вес для взаимодействия со статьёй (в соответствии с приведёнными выше весами). Вычислите среднее значение для полученного признака.

In [24]:
interactions_data['event_num'] = interactions_data['eventType'].apply(lambda x: event_type[x])
interactions_data['event_num'].mean()

1.2362885828078327

Чтобы получить хоть какую-то информацию, на которую можно будет опираться, оставьте только тех пользователей, которые взаимодействовали хотя бы с пятью статьями. Сколько всего таких пользователей?

Оставим только те взаимодействия, которые касаются только отфильтрованных пользователей (то есть тех, которые взаимодействовали как минимум с пятью статьями). Сколько всего таких взаимодействий?

In [38]:
freq_users = interactions_data.groupby(by='personId', as_index=True)['contentId'].nunique()
freq_users = freq_users[freq_users >= 5]
interactions_data = interactions_data[interactions_data['personId'].isin(freq_users.index)]
interactions_data.shape[0], freq_users.shape[0]

(69868, 1140)

Сейчас каждое отдельное взаимодействие пользователя со статьёй выделено в отдельную запись, то есть пользователь мог просмотреть статью, лайкнуть и прокомментировать её, и всё это отразилось в трёх действиях. Давайте для удобства соединим все эти действия в некоторый коэффициент, который будет отражать интерес пользователя к статье. Так как каждому возможному действию мы ранее уже присвоили вес, то, по сути, нам нужно просто сложить все действия. Однако полученное число будет увеличиваться с количеством действий, и будет очень большой разброс возможных значений. В таких случаях обычно логарифмируют полученный результат с помощью следующей функции:

In [43]:
def smooth_user_preference(x):
    return np.log2(1+x)

Примените упомянутое выше преобразование для логарифмирования к сумме весов для взаимодействия пользователя с каждой конкретной статьёй. Также сохраните для каждой пары «пользователь — статья» значение времени последнего взаимодействия.

Найдите среднее по признаку с получившимися временными отсечками.

In [54]:
users_sum_event = interactions_data.groupby(
    by=['personId', 'contentId'],
    as_index=False
)['event_num'].sum()
users_sum_event['event_num'] = users_sum_event['event_num'].apply(smooth_user_preference)
users_sum_event['last_timestamp'] = interactions_data.groupby(by=['personId', 'contentId'], as_index=False)['timestamp'].max()['timestamp']
users_sum_event['last_timestamp'].mean()

1470605340.0403006

Разделите данные на обучающую и тестовую выборки, выбрав в качестве временной отсечки значение 1475519545. Значение отсечки включите в тестовую выборку.

In [56]:
separator = 1475519545
train, test = users_sum_event[users_sum_event['last_timestamp'] < separator], users_sum_event[users_sum_event['last_timestamp'] >= separator]
train.shape[0], test.shape[0]

(29325, 9781)

Для удобства дальнейшего измерения качества рекомендаций преобразуйте данные так, чтобы получить таблицу в формате, где строка соответствует пользователю, а столбцы будут истинными предпочтениями и рекомендациями в формате списков. На место пустых ячеек поместите пустые списки.



In [59]:
final_df = (
    train.reset_index()
    .groupby('personId')['contentId'].agg(lambda x: list(x))
    .reset_index()
    .rename(columns={'contentId': 'true_train'})
    .set_index('personId')
)

final_df['true_test'] = (
    test.reset_index()
    .groupby('personId')['contentId'].agg(lambda x: list(x))
)

final_df['true_test'] = [ [] if x is np.NaN else x for x in final_df['true_test'] ]
final_df.head()

,true_train,true_test
personId,,
-1007001694607905623,"[-5065077552540450930, -793729620925729327]","[-6623581327558800021, 1469580151036142903, 72..."
-1032019229384696495,"[-1006791494035379303, -1039912738963181810, -...","[-1415040208471067980, -2555801390963402198, -..."
-108842214936804958,"[-1196068832249300490, -133139342397538859, -1...","[-2780168264183400543, -3060116862184714437, -..."
-1130272294246983140,"[-1150591229250318592, -1196068832249300490, -...","[-1606980109000976010, -1663441888197894674, -..."
-1160159014793528221,"[-133139342397538859, -387651900461462767, 377...",[-3462051751080362224]


In [63]:
popular = (
    train
    .groupby('contentId')
    .event_num.sum().reset_index()
    .sort_values('event_num', ascending=False)
    .contentId.values
)
popular[0]

'-6783772548752091658'

In [72]:
top_k = 10
 
final_df['popular'] = (
    final_df.true_train
    .apply(
        lambda x:
        popular[~np.in1d(popular, x)][:top_k]
    )
)

def precision(column):
    return (final_df.apply(lambda row:
            len(set(row['true_test']).intersection(set(row[column]))) /
            min(len(row['true_test']) + 0.001, 10.0),axis=1)).mean()
    
precision('popular')

0.006454207722621089